#import

In [215]:
import numpy as np
import pandas as pd
import librosa

from sklearn.model_selection import train_test_split
from scipy.stats import skew

from sklearn.multiclass import  OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from keras.utils import np_utils
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsOneClassifier


import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LayerNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score

資料前處理

In [217]:
# 資料判斷
df_csv = pd.read_csv("Training Dataset/training datalist.csv")
print("資料資訊")
df_csv.info()

資料資訊
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         1000 non-null   object 
 1   Sex                        1000 non-null   int64  
 2   Age                        1000 non-null   int64  
 3   Disease category           1000 non-null   int64  
 4   Narrow pitch range         1000 non-null   int64  
 5   Decreased volume           1000 non-null   int64  
 6   Fatigue                    1000 non-null   int64  
 7   Dryness                    1000 non-null   int64  
 8   Lumping                    1000 non-null   int64  
 9   heartburn                  1000 non-null   int64  
 10  Choking                    1000 non-null   int64  
 11  Eye dryness                1000 non-null   int64  
 12  PND                        1000 non-null   int64  
 13  Smoking                    1000 non-null   i

In [218]:
# 挑選出要訓練的Disease category 1、2、3
df_csv = df_csv.loc[df_csv['Disease category'].isin([1, 2, 3, 4, 5]), ['ID', 'Disease category']]

# 在dataframe中加入要訓練的音檔路徑
df_csv['wav_path'] = df_csv['ID'].map("./Training Dataset/training_voice_data/{}.wav".format)

print("Disease category in source_df :",df_csv['Disease category'].unique())
print("source_df :\n", df_csv)

Disease category in source_df : [1 2 3 5 4]
source_df :
           ID  Disease category  \
0    1202f15                 1   
1    0600ve0                 2   
2    1001o7l                 2   
3    1201c1t                 1   
4    0402jvt                 1   
5    1102ial                 1   
6    0K024ja                 3   
7    00003h4                 2   
8    1101t8u                 1   
9    11019sl                 1   
10   1100jkh                 1   
11   0801ifi                 1   
12   0T000fb                 5   
13   1101fh2                 1   
14   0402svh                 4   
15   0T0018b                 3   
16   1300ams                 3   
17   1102hc2                 1   
18   0T00ufl                 5   
19   11014l7                 1   
20   0T01uvq                 3   
21   1202n63                 1   
22   12007bm                 1   
23   1102vjo                 3   
24   1101ko5                 1   
25   0402sfe                 2   
26   0100o4g             

In [219]:
# 定義函數
def audio_to_mfccs(filename, sample_rate=44100, offset=0, duration=None):
    # 讀取音訊檔案，並設定取樣率、起始時間、及持續時間
    voice, sample_rate = librosa.load(
        filename, sr=sample_rate, offset=offset, duration=duration
    )

    # 將時間值轉換為 FFT 與 hop length 所需的框架數 (以取樣點表示)
    n_fft = int(16/1000 * sample_rate)  # 將 16 毫秒轉換為取樣點
    hop_length = int(8/1000 * sample_rate)  # 將 8 毫秒轉換為取樣點

    # 計算音訊數據的 MFCC 特徵
    mfcc_feature = librosa.feature.mfcc(
        y=voice, sr=sample_rate, n_mfcc=13, n_fft=n_fft, hop_length=hop_length)

    # 計算 MFCC 的一階和二階差分特徵
    delta_mfcc_feature = librosa.feature.delta(mfcc_feature)

    # 將原始 MFCC 特徵和差分特徵串聯起來，得到所有幀的特徵向量
    mfccs = np.concatenate((mfcc_feature, delta_mfcc_feature))
    mfccs_features = np.transpose(mfccs)  # 將矩陣轉置，使每行代表一個幀

    # 返回特徵向量
    return mfccs_features

In [220]:

training_id = df_csv['ID'].tolist()
training_data = pd.DataFrame()
for id in training_id:
    
    mfccs_feature = audio_to_mfccs(df_csv[df_csv['ID']==id]['wav_path'].values[0])
    df = pd.DataFrame()
    for i in range(26):
        df_i = pd.DataFrame(np.array(mfccs_feature[0][i]).reshape(1,-1))
        df = pd.concat([df, df_i], axis=1)
        
    # print("id :",id, ", number of frames :", df.shape[0])
    # 訓練資料標記
    label = df_csv[df_csv['ID']==id]['Disease category'].values[0]
    if label==1:
        df['c1'] = 1; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 0
    elif label==2:
        df['c1'] = 0; df['c2'] = 1; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 0
    elif label==3:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 1 ; df['c4'] = 0; df['c5'] = 0
    elif label==4:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 1; df['c5'] = 0
    elif label==5:
        df['c1'] = 0; df['c2'] = 0; df['c3'] = 0 ; df['c4'] = 0; df['c5'] = 1
    else:
        df['c1'] = np.nan; df['c2'] = np.nan; df['c3'] = np.nan; df['c4'] = np.nan; df['c5'] = np.nan

    training_data = pd.concat([training_data, df])
    
print("training_data.shape :", training_data.shape)

x_train = training_data.iloc[:, :-5]
y_train = training_data.iloc[:, -5:]
print("x_train.shape, y_train.shape :", x_train.shape, y_train.shape)
print("y_train.columns :", y_train.columns.tolist())


training_data.shape : (1000, 31)
x_train.shape, y_train.shape : (1000, 26) (1000, 5)
y_train.columns : ['c1', 'c2', 'c3', 'c4', 'c5']


In [221]:
x = x_train.to_numpy()
y = y_train.to_numpy()
print(y)
print(x)
print(y.shape)
print(x.shape)
y = np.argmax(y, axis=1)

print(y)

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 ...
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]]
[[-5.53074524e+02  1.48494080e+02  1.46729660e+01 ... -3.93367976e-01
  -3.19579206e-02  1.06491983e+00]
 [-2.22018585e+02  1.32414246e+02 -1.35683060e+00 ... -9.75926936e-01
  -3.62859637e-01  6.54560566e-01]
 [-3.39410553e+02  1.86702805e+02 -2.79492817e+01 ... -4.88830119e-01
  -7.36986995e-01  4.74724650e-01]
 ...
 [-2.79279510e+02  1.13782791e+02 -2.01488895e+01 ... -6.52139187e-01
  -1.33095014e+00  1.11008868e-01]
 [-4.17048859e+02  1.23644379e+02  3.54444265e+00 ... -1.04734026e-01
   1.49622679e+00 -7.80113876e-01]
 [-3.73614166e+02  1.93103134e+02  2.38179951e+01 ...  3.70338380e-01
   1.18615009e-01  1.16457713e+00]]
(1000, 5)
(1000, 26)
[0 1 1 0 0 0 2 1 0 0 0 0 4 0 3 2 2 0 4 0 2 0 0 2 0 1 1 0 4 0 4 1 0 0 2 3 1
 1 0 4 1 1 1 0 4 0 2 0 2 0 0 1 0 2 3 0 0 0 0 0 2 1 3 0 0 0 0 3 2 0 0 0 3 2
 2 1 0 0 0 2 0 0 2 3 0 0 4 2 2 0 1 0 2 2 2 1 0 1 1 0 1 1 0 0 2 1 0 1 2 0 2
 0 1 0 0 0 0 1 0 0 0 0 0 1 2 0 0 0

In [222]:
for i in range(len(y)):
    y[i] += 1

In [223]:
new_x = np.zeros((1000, 6))
for i in range(1000):
    # 計算最大值
    max_a = np.max(x[i])

    # 計算最小值
    min_a = np.min(x[i])

    # 計算平均數
    mean_a = np.mean(x[i])

    # 計算中位數
    median_a = np.median(x[i])

    # 計算標準差
    std_a = np.std(x[i])

    # 計算偏度
    skew_a = skew(x[i])

    # 轉換矩陣為一個包含六個元素的陣列
    new_x[i] = np.array([max_a, min_a, mean_a, median_a, std_a, skew_a])
print(new_x)

[[ 1.48494080e+02 -5.53074524e+02 -1.63179493e+01 -5.11780977e-01
   1.11396294e+02 -4.17364942e+00]
 [ 1.32414246e+02 -2.22018585e+02 -6.04307556e+00 -9.04636979e-01
   5.09200935e+01 -2.15934679e+00]
 [ 1.86702805e+02 -3.39410553e+02 -1.00040083e+01 -6.12908542e-01
   7.62460938e+01 -2.44126632e+00]
 ...
 [ 1.13782791e+02 -2.79279510e+02 -9.72987843e+00 -1.34194613e+00
   5.88886871e+01 -3.32960002e+00]
 [ 1.23644379e+02 -4.17048859e+02 -1.30127821e+01 -2.56308377e-01
   8.47630157e+01 -3.99959885e+00]
 [ 1.93103134e+02 -3.73614166e+02 -9.09355068e+00 -3.75599675e-02
   8.26008835e+01 -2.73785199e+00]]


In [224]:
x_train2, x_test ,y_train2, y_test = train_test_split(new_x, y, test_size=0.8, random_state=42)

clf = OneVsOneClassifier(SVC(kernel="linear"))

clf.fit(x_train2,y_train2)

y_pred = clf.predict(x_test)


In [ ]:
print(y_pred)